# Dockerizing a Flask model

### Model training

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pickle

In [16]:
iris = load_iris()
ada = AdaBoostClassifier(n_estimators=5)

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.1)
model = ada.fit(X_train, y_train)

print('Model Accuracy:',ada.score(X_train, y_train))
print('Validation Accuracy:',ada.score(X_test, y_test))

Model Accuracy: 0.9481481481481482
Validation Accuracy: 0.9333333333333333


In [17]:
pickle.dump(model, open('model.pkl', 'wb'))
model1 = pickle.load(open('model.pkl', 'rb'))

In [42]:
model1.predict([[6.5, 3. , 5.5, 1.8]])

array([2])

### Model deployment

In [ ]:
#app.py

import pickle
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/api/', methods=['POST'])
def predict_category():
    
    #Get input from POST and preprocess
    data = request.get_json()
    inp = [[i for i in data.values()]]
    
    #Load and predict using model
    model = pickle.load(open('model.pkl', 'rb'))
    output = model.predict(inp)
    
    #Return output
    outdict = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
    return jsonify(category=outdict.get(output[0]))

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0',port=5000)//

In [ ]:
## IN POSTMAN - 

#URL
http://localhost:5000/api/

#BODY (raw, JSON)
{
"sepal_length":6.5,
"sepal_width":3.0,
"petal_length":5.5,
"petal_width":1.8
}

#OUTPUT
{
    "category": "virginica"
}

In [ ]:
#REQIREMENTS 

requirements.txt

Flask==1.0.2
scikit-learn==0.23.2

In [44]:
## DOCKER BUILD

#-----------------------

#IN DOCKERFILE (Dockerfile.txt)
FROM continuumio/miniconda3

WORKDIR /app

COPY . /app

EXPOSE 5000
CMD ["python", "api.py"]

#-----------------------

#DOCKER BUILD
$ docker build -t irisapp -f Dockerfile.txt .

#output
Sending build context to Docker daemon  17.41kB
Step 1/6 : FROM continuumio/miniconda3
 ---> b4adc22212f1
Step 2/6 : WORKDIR /app
 ---> Using cache
 ---> 24e9a13027c3
Step 3/6 : COPY . /app
 ---> bda82ae7733c
Step 4/6 : RUN pip install -r requirements.txt
 ---> Running in c711eaf981e7
Collecting Flask==1.0.2
  Downloading Flask-1.0.2-py2.py3-none-any.whl (91 kB)
Collecting scikit-learn==0.23.2
  Downloading scikit_learn-0.23.2-cp37-cp37m-manylinux1_x86_64.whl (6.8 MB)
Successfully installed Flask-1.0.2 scikit-learn-0.23.2
Removing intermediate container c711eaf981e7
 ---> 5e319e7da4f5
Step 5/6 : EXPOSE 5000
 ---> Running in a8b2a33abafd
Removing intermediate container a8b2a33abafd
 ---> f0beae72b604
Step 6/6 : CMD ["python", "app.py"]
 ---> Running in ed78f6606d7a
Removing intermediate container ed78f6606d7a
 ---> 217d750d4ea5
Successfully built 217d750d4ea5
Successfully tagged irisapp:latest

##----------------------

$ docker images

REPOSITORY               TAG                 IMAGE ID            CREATED             SIZE
irisapp                  latest              217d750d4ea5        4 minutes ago       655MB
continuumio/miniconda3   latest              b4adc22212f1        5 months ago        429MB

In [ ]:
## DOCKER RUN

$ docker run -p 5000:5000 irisapp
    
* Serving Flask app "app" (lazy loading)
* Environment: production
  WARNING: Do not use the development server in a production environment.
  Use a production WSGI server instead.
* Debug mode: off
* Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)